In [28]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [29]:
# Pandas
import pandas as pd

# SQL Alchemy

from sqlalchemy import create_engine

import psycopg2
from config import connect

conn_string = connect
conn = psycopg2.connect(conn_string)